## Using Data From Virtuoso in Tensorflow (SQL Query)

In [52]:
import pandas as pd
import pyodbc
import tensorflow as tf
import datetime
import numpy as np

SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

### Set Connection, and Create Dataframe

In [72]:
cnxn = pyodbc.connect("DSN=Local Virtuoso;UID=dba;pwd=dba")

q = 'SELECT * FROM "tensorflow"."diabetes".data'

d = pd.read_sql_query(q,cnxn)

d.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [14]:
df = pd.DataFrame(d)

outcome = df.pop("Outcome")

df.head()



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [15]:
feat_names = df.columns.tolist()
feats = df[feat_names]


### Convert to Tensor and continue in TensorFlow

In [16]:
print(tf.convert_to_tensor(feats))

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(feats)
normalizer(feats.iloc[:3])

2021-11-30 11:12:35.345980: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


tf.Tensor(
[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  2.    122.     70.    ...  36.8     0.34   27.   ]
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]], shape=(767, 8), dtype=float64)


2021-11-30 11:12:35.708140: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 0.6387271 ,  0.84705454,  0.14960383,  0.90778947, -0.6935593 ,
         0.20362139,  0.4676379 ,  1.4246367 ],
       [-0.8458293 , -1.1243613 , -0.16038112,  0.5315605 , -0.6935593 ,
        -0.6842581 , -0.36549374, -0.1917781 ],
       [ 1.2325497 ,  1.9422855 , -0.26370946, -1.2868794 , -0.6935593 ,
        -1.1028302 ,  0.6034746 , -0.10670363]], dtype=float32)>

In [104]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

model = get_basic_model()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(feats, outcome, epochs=10, batch_size=BATCH_SIZE)

2021-11-30 12:58:40.771449: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-30 12:58:40.771464: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-30 12:58:40.771561: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/10
384/384 [==============================] - 0s 616us/step - loss: 1.1714 - accuracy: 0.6128
Epoch 2/10
384/384 [==============================] - 0s 609us/step - loss: 0.5298 - accuracy: 0.7471
Epoch 3/10
384/384 [==============================] - 0s 610us/step - loss: 0.4912 - accuracy: 0.7705
Epoch 4/10
384/384 [==============================] - 0s 615us/step - loss: 0.4718 - accuracy: 0.7757
Epoch 5/10
384/384 [==============================] - 0s 613us/step - loss: 0.4652 - accuracy: 0.7653
Epoch 6/10
384/384 [==============================] - 0s 614us/step - loss: 0.4580 - accuracy: 0.7810
Epoch 7/10
384/384 [==============================] - 0s 612us/step - loss: 0.4509 - accuracy: 0.7875
Epoch 8/10
384/384 [==============================] - 0s 621us/step - loss: 0.4489 - accuracy: 0.7810
Epoch 9/10
384/384 [==============================] - 0s 620us/step - loss: 0.4392 - accuracy: 0.7823
Epoch 10/10
384/384 [==============================] - 0s 627us/step - loss: 0.439

In [112]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


## Predictions

In [123]:
sample = [[6,148,72,35,0,33.6,.627,50],[6,148,72,35,0,33.6,.627,60],[1,85,66,29,0,26.6,0.351,55],[1,185,66,29,0,26.6,0,55]]
prediction = model.predict(sample)
print(prediction)

classes = np.argmax(prediction, axis = 1)
print(classes)

[[3.21099550e-01 6.76772356e-01 1.11034315e-04 5.50576544e-04
  2.13246778e-04 6.25148474e-04 1.44753780e-04 1.03056584e-04
  3.27100803e-04 5.32370614e-05]
 [3.70464563e-01 6.29046917e-01 2.02863412e-05 1.09953129e-04
  3.20507788e-05 1.81329451e-04 2.89274758e-05 2.44750154e-05
  8.22258371e-05 9.15512283e-06]
 [8.77539098e-01 1.22305572e-01 6.64299932e-06 5.23731433e-06
  5.48891467e-06 6.06262547e-05 8.12885719e-06 3.57990830e-05
  1.74381457e-05 1.60761447e-05]
 [2.55893499e-01 7.42153645e-01 7.50573890e-05 5.17058070e-04
  2.45227478e-04 4.51016560e-04 1.34911723e-04 8.73098979e-05
  4.07310145e-04 3.49391667e-05]]
[1 1 0 1]
